# Data Gathering

This notebook gathers the meta-data for the evaluation. It runs the various respiratory extraction methods.

In [1]:
import json
import os

import respiratory_extraction.dataset as repository

dataset = repository.from_default()
subjects = dataset.get_subjects()
subjects

['Proband05', 'Proband16']

In [2]:
# The scenarios in which the subjects were recorded
# scenarios = dataset.get_scenarios()
# scenarios = [
#     '101_natural_lighting',
#     # '102_artificial_lighting',
#     '103_abrupt_changing_lighting',
#     '104_dim_lighting_auto_exposure',
#     # '106_green_lighting',
#     # '107_infrared_lighting',
#     '201_shouldercheck',
#     '202_scale_movement',
#     '203_translation_movement',
#     # '204_writing'
# ]
scenarios = [
    '101_natural_lighting',
]
scenarios

['101_natural_lighting']

In [3]:
from datetime import datetime

hyperparameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
    'filter_lowpass': 0.1,
    'filter_highpass': 0.6,
}

evaluation_metadata = {
    'start_time': datetime.now(),
    'subjects': subjects,
    'scenarios': scenarios,
}

In [7]:
import json
import respiratory_extraction.utils as utils

from respiratory_extraction.models.signal_extraction import pixel_intensity, optical_flow

extracted_signals = []

faulty_scenarios = []

lowpass = hyperparameters['filter_lowpass']
highpass = hyperparameters['filter_highpass']

for subject in subjects:
    for idx, scenario in enumerate(scenarios):
        print(f'Processing {subject} - {scenario}')

        frames, params = dataset.read_video_gray(subject, scenario)

        faces = utils.detect_faces(frames[0])
        if len(faces) != 1:
            print(f'No face detected in {subject} - {scenario}')
            faulty_scenarios.append((subject, scenario))
            continue

        # Calculate the region of interest (ROI) based on the face
        roi = utils.roi_from_face(faces[0])

        #
        # Calculate the average pixel intensity
        #

        pi_start = datetime.now()
        pi_signal = pixel_intensity.average_pixel_intensity(frames, roi=roi)
        extracted_signals.append({
            'subject': subject,
            'scenario': scenario,
            'method': 'pixel_intensity',
            'parameter': {
                'roi': list(roi),
            },
            'execution_time': datetime.now() - pi_start,
            'sampling_rate': params.fps,
            'signal': pi_signal.tolist(),
        })

        #
        # Calculate the optical flow without cgof
        # 

        of_raw_start = datetime.now()
        of_signal_raw = optical_flow.extract_signal(
            frames,
            roi=roi,
            quality_level=hyperparameters['quality_level'],
            quality_level_rv=hyperparameters['quality_level_rv'],
            use_cgof=False,
        )
        extracted_signals.append({
            'subject': subject,
            'scenario': scenario,
            'method': 'optical_flow',
            'parameter': {
                'roi': list(roi),
                'use_cgof': False
            },
            'execution_time': datetime.now() - of_raw_start,
            'sampling_rate': params.fps,
            'signal': of_signal_raw.tolist(),
        })

        #
        # Calculate the optical flow with cgof
        #

        of_cgof_start = datetime.now()
        of_signal_raw = optical_flow.extract_signal(
            frames,
            roi=roi,
            quality_level=hyperparameters['quality_level'],
            quality_level_rv=hyperparameters['quality_level_rv'],
            use_cgof=True,
        )

        extracted_signals.append({
            'subject': subject,
            'scenario': scenario,
            'method': 'optical_flow',
            'parameter': {
                'roi': list(roi),
                'use_cgof': True
            },
            'execution_time': datetime.now() - of_cgof_start,
            'sampling_rate': params.fps,
            'signal': of_signal_raw.tolist(),
        })

        # Garbage collect the frames
        del frames

Processing Proband05 - 101_natural_lighting


Reading video:   0%|          | 0/3600 [00:00<?, ?it/s]

Processing Proband16 - 101_natural_lighting


Reading video:   0%|          | 0/3600 [00:00<?, ?it/s]

In [9]:
faulty_scenarios

[]

In [10]:
evaluation_metadata['faulty_scenarios'] = faulty_scenarios
evaluation_metadata['end_time'] = datetime.now()

In [11]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

,subject,scenario,method,parameter,execution_time,sampling_rate,signal
0,Proband05,101_natural_lighting,pixel_intensity,"{'roi': [272, 395, 129, 64], 'use_cgof': False}",0 days 00:00:00.027954,30,"[74.06964631782945, 74.05995639534883, 74.0652..."
1,Proband05,101_natural_lighting,optical_flow,"{'roi': [272, 395, 129, 64], 'use_cgof': False}",0 days 00:00:02.222249,30,"[549.913471117982, 549.9467925031414, 549.9584..."
2,Proband05,101_natural_lighting,optical_flow,"{'roi': [272, 395, 129, 64], 'use_cgof': False}",0 days 00:00:02.417030,30,"[549.8965005721562, 549.9339159204322, 549.943..."
3,Proband16,101_natural_lighting,pixel_intensity,"{'roi': [251, 430, 120, 60], 'use_cgof': False}",0 days 00:00:00.019499,30,"[25.891666666666666, 26.140166666666666, 25.74..."
4,Proband16,101_natural_lighting,optical_flow,"{'roi': [251, 430, 120, 60], 'use_cgof': False}",0 days 00:00:02.504119,30,"[561.0121719456519, 560.7956244465958, 560.667..."


In [12]:
evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'signals')
if not os.path.exists(evaluation_dir):
    os.makedirs(evaluation_dir)

# Save the extracted_signals as a JSON
json_path = os.path.join(evaluation_dir, 'metadata.json')
utils.write_json(json_path, extracted_signals)

# Save the evaluation dataframe
csv_path = os.path.join(evaluation_dir, 'metadata.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(evaluation_dir, 'parameters.json')
utils.write_json(json_path, evaluation_metadata)